In [1]:
# Importing modules
import pandas as pd
import numpy as np
from config import config

# Reading the raw dataset
raw_dataset = pd.read_csv(config['path_raw'] + 'TACE_LR.csv')
print("The raw_dataset has the following shape: {}.".format(raw_dataset.shape))
raw_dataset.describe()

The raw_dataset has the following shape: (5073, 22).


,ID,state,time,treatment_before,gender,age,Etiology,ECOG,tumor_number,tumor_size,...,ALB,TBIL,AST,ALT,PLT,INR,BUN,Cr,WBC,HGB
count,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,...,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000
mean,5054.965109,0.456535,32.353119,0.046521,0.849793,54.967988,1.064065,0.272620,1.777055,6.478766,...,40.522624,16.853966,54.996409,54.959300,143.088602,1.061043,5.452260,74.631538,5.197461,136.292104
std,2291.592780,0.498156,22.551936,0.210631,0.357309,11.913047,0.244892,0.445351,1.413436,3.767633,...,4.840245,7.621541,77.919350,77.191351,77.455903,0.115742,1.927816,20.905981,3.353453,20.091716
min,1001.000000,0.000000,1.033333,0.000000,0.000000,18.000000,1.000000,0.000000,1.000000,0.700000,...,20.000000,0.100000,1.000000,2.000000,11.000000,0.690000,0.900000,3.000000,1.000000,33.300000
25%,2736.000000,0.000000,13.466667,0.000000,1.000000,46.000000,1.000000,0.000000,1.000000,3.500000,...,37.200000,11.500000,29.000000,25.000000,86.000000,0.990000,4.310000,64.000000,3.400000,125.000000
50%,5664.000000,0.000000,28.166667,0.000000,1.000000,55.000000,1.000000,0.000000,1.000000,5.500000,...,40.500000,15.300000,41.000000,37.000000,134.000000,1.050000,5.300000,73.088394,4.640000,138.000000
75%,7018.000000,1.000000,46.533333,0.000000,1.000000,64.000000,1.000000,1.000000,2.000000,8.800000,...,43.800000,20.500000,58.614578,60.000000,185.000000,1.130000,6.200000,83.000000,6.269280,150.000000
max,8427.000000,1.000000,141.233333,1.000000,1.000000,91.000000,2.000000,1.000000,12.000000,22.600000,...,75.400000,84.000000,2900.000000,2700.000000,634.000000,3.100000,51.000000,824.000000,75.000000,207.000000


In [ ]:
# Defining the time and event column
time_column = 'time'
event_column = 'state'

# encode, drop...
dataset = raw_dataset.drop(['ID'], axis=1)
dataset_with_id = raw_dataset

# Defining the modeling features
features_all = np.setdiff1d(dataset.columns, ['time', 'state']).tolist()
features_select = ["tumor_size","therapy", "AFP", "AST", "tumor_number", "ECOG", "ALB", "HGB", "INR"]

# Checking for null values
N_null = sum(dataset[features_all].isnull().sum())
print("The dataset contains {} null values".format(N_null)) #0 null values

# Removing duplicates if there exist
N_dupli = sum(dataset.duplicated(keep='first'))
N_dupli_list = dataset.index[dataset.duplicated(keep='first') == True].tolist()
print(N_dupli_list)
dataset = dataset.drop_duplicates(keep='first').reset_index(drop=True)
dataset_with_id = dataset_with_id.drop(N_dupli_list, axis=0).reset_index(drop=True)
print("The dataset contains {} duplicates".format(N_dupli))

print(dataset.shape)
print(dataset_with_id.shape)

In [ ]:
dataset = dataset_with_id

# Building training and testing sets
from sklearn.model_selection import train_test_split
index_train, index_test = train_test_split( range(dataset.shape[0]), test_size = config['valid_ratio'], random_state = config['seed'], shuffle = True)
data_train = dataset.loc[index_train].reset_index( drop = True )
data_test  = dataset.loc[index_test].reset_index( drop = True )

# Print out the data size.
print("The training set has the following shape: {}.".format(data_train.shape))
print("The testing set has the following shape: {}.".format(data_test.shape))

# save the data
data_train.to_csv(config['path_processed'] + 'data_train.csv', index = False)
data_test.to_csv(config['path_processed'] + 'data_test.csv', index = False)